In [1]:
import sys
sys.path.insert(0, '../../toyaikit/')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from agents import Agent, Runner, SQLiteSession, function_tool

## Coding agent

In [4]:
import os
import shutil

def copy_project_template(project_folder: str) -> str:
    """
    copy the template into the project folder

    arguments: project_folder - should be a lowercase string with underscores
    """
    if not project_folder:
        print("Project name cannot be empty.")
        return

    if os.path.exists(project_folder):
        print(f"Directory '{project_folder}' already exists. Please choose a different name or remove the existing directory.")
        return

    shutil.copytree('django_template', project_folder)
    print(f"Django template copied to '{project_folder}' directory.")

    return project_folder

In [5]:
project_name = copy_project_template('todo-agents')

Directory 'todo-agents' already exists. Please choose a different name or remove the existing directory.


In [6]:
project_name = 'todo-agents'

In [7]:
import tools
from pathlib import Path

In [8]:
agent_tools = tools.AgentTools(Path(project_name))

coding_agent_tools_list = [
    function_tool(agent_tools.execute_bash_command),
    function_tool(agent_tools.read_file),
    function_tool(agent_tools.search_in_files),
    function_tool(agent_tools.see_file_tree),
    function_tool(agent_tools.write_file)
]

In [9]:
DEVELOPER_PROMPT = """
You are a coding agent. Your task is to modify the provided Django project template
according to user instructions. You don't tell the user what to do; you do it yourself using the 
available tools. First, think about the sequence of steps you will do, and then 
execute the sequence.
Always ensure changes are consistent with Django best practices and the project’s structure.

## Project Overview

The project is a Django 5.2.4 web application scaffolded with standard best practices. It uses:
- Python 3.8+
- Django 5.2.4 (as specified in pyproject.toml)
- uv for Python environment and dependency management
- SQLite as the default database (see settings.py)
- Standard Django apps and a custom app called myapp
- HTML templates for rendering views
- TailwindCSS for styling

## File Tree


├── .python-version
├── README.md
├── manage.py
├── pyproject.toml
├── uv.lock
├── myapp/
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations/
│   │   └── __init__.py
│   ├── models.py
│   ├── templates/
│   │   └── home.html
│   ├── tests.py
│   └── views.py
├── myproject/
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── templates/
    └── base.html

## Content Description

- manage.py: Standard Django management script for running commands.
- README.md: Setup and run instructions, including use of uv for dependency management.
- pyproject.toml: Project metadata and dependencies (Django 5.2.4).
- uv.lock: Lock file for reproducible Python environments.
- .python-version: Specifies the Python version for the project.
- myapp/: Custom Django app with models, views, admin, tests, and a template (home.html).
  - migrations/: Contains migration files for database schema.
- myproject/: Django project configuration (settings, URLs, WSGI/ASGI entrypoints).
  - settings.py: Configures installed apps, middleware, database (SQLite), templates, etc.
- templates/: Project-level templates, including base.html.

You have full access to modify, add, or remove files and code within this structure using your available tools.


## Additional instructions

- Don't execute "runserver", but you can execute other commands to check if the project is working.
- Make sure you use Tailwind styles for making the result look beatiful
- Use pictograms and emojis when possible. Font-awesome is awailable
- Avoid putting complex logic to templates - do it on the server side when possible
"""

In [10]:
coding_agent = Agent(
    name="CodingAgent",
    instructions=DEVELOPER_PROMPT,
    tools=coding_agent_tools_list,
    model='gpt-4o-mini'
)

In [23]:
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

In [24]:
from toyaikit.chat import IPythonChatInterface
interface = IPythonChatInterface()

In [25]:
runner = OpenAIAgentsSDKRunner(chat_interface=interface, agent=coding_agent)

In [22]:
await runner.run()

You: todo list


You: stop


Chat ended.
